<a href="https://colab.research.google.com/github/ekaterina533/dataset/blob/main/%D0%9F%D0%B5%D1%80%D0%B5%D0%BD%D0%BE%D1%81%20%D1%82%D0%B5%D1%81%D1%82%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install ldap3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.2/432.2 kB 11.8 MB/s eta 0:00:00


In [3]:
%%writefile core.py
import os
import logging
from logging.handlers import RotatingFileHandler
import json

class Config(object):

    def __init__(self, log_level: str='DEBUG', path_to_export_file: str='./', ldap_server: str=None, ldap_user: str=None, ldap_password: str=None, ldap_bind: str=None, email_login: str='test@avt-stv.ru', email_password: str='ddd', email_server: str='avt-stv.ru', mail_send_to: str='it@avt-stv.ru', ignore_on_departament_change_dn_group: list=[]):
        self.log_level = log_level
        self.path_to_export_file = path_to_export_file
        self.ldap_server = ldap_server
        self.ldap_user = ldap_user
        self.ldap_password = ldap_password
        self.ldap_bind = ldap_bind
        self.email_login = email_login
        self.email_password = email_password
        self.email_server = email_server
        self.mail_send_to = mail_send_to
        self.ignore_on_departament_change_dn_group = ignore_on_departament_change_dn_group

    def read_conf(cls):
        if not os.path.exists('config.json'):
            data = json.dumps(cls.__dict__)
            with open('config.json', 'w') as f:
                f.write(data)
        with open('config.json', encoding='utf-8') as f:
            data = json.load(f)
            for item in dir(cls):
                if not callable(getattr(cls, item)) and (not item.startswith('__')):
                    try:
                        setattr(cls, item, data[item])
                    except KeyError:
                        pass
            return cls
config = Config().read_conf()
logging.basicConfig(handlers=[RotatingFileHandler('log', mode='a', maxBytes=5242880, backupCount=0, encoding='utf-8', delay=0), logging.StreamHandler()], level=logging.getLevelName(config.log_level), format='%(asctime)s.%(msecs)03d %(levelname)s:%(message)s', datefmt='%d.%m.%Y %H:%M:%S')

Writing core.py


In [1]:
%%writefile onec.py

from core import config, logging
from objects import File_user_object
import json

def import_users_from_file(ad_ous):
    logging.info('Ищем пользователей в 1C')
    users = []
    f = open(config.path_to_export_file, encoding='utf-8')
    data = json.load(f)
    for item in data:
        name = item['ФИО'].split(' ')
        try:
            first_name = name[1]
        except IndexError:
            first_name = None
        try:
            last_name = name[0]
        except IndexError:
            last_name = None
        try:
            middle_name = name[2]
        except IndexError:
            middle_name = None
        director_name = item['Руководитель'].split(' ')
        try:
            director_first_name = director_name[1]
        except IndexError:
            director_first_name = None
        try:
            director_last_name = director_name[0]
        except IndexError:
            director_last_name = None
        try:
            director_middle_name = director_name[2]
        except IndexError:
            director_middle_name = None
        code = int(item['Код'])
        division = item['ПодразделениеУчасток']
        departament = item['Подразделение']
        post = item['Должность']
        number = item['Телефон']
        status = item['Статус']
        if status == 'работает':
            status = True
        else:
            status = False
        departament_code = int(item['КодПодразделения'])
        division_code = int(item['КодПодразделенияУчастка'])
        director_code = int(item['КодРуководителя'])
        division_OU = None
        for ad_ou in ad_ous:
            if ad_ou.code == division_code:
                division_OU = ad_ou
                break
        departament_OU = None
        for ad_ou in ad_ous:
            if ad_ou.code == departament_code:
                departament_OU = ad_ou
                break
        users.append(File_user_object(departament_OU=departament_OU, division_OU=division_OU, departament_code=departament_code, division_code=division_code, director_code=director_code, first_name=first_name, last_name=last_name, middle_name=middle_name, director_first_name=director_first_name, director_last_name=director_last_name, director_middle_name=director_middle_name, code=code, division=division, departament=departament, post=post, number=number, works=status))
    logging.info(f'Найдено {len(users)} пользователей')
    return users

Writing onec.py


In [17]:
%%writefile gost_translate.py

def gost_translate(first_name, middle_name, last_name):
    TRANSLITERATE_DICT = {'а':'a','б':'b','в':'v','г':'g','д':'d','е':'e','ё':'yo',
      'ж':'zh','з':'z','и':'i','й':'i','к':'k','л':'l','м':'m','н':'n',
      'о':'o','п':'p','р':'r','с':'s','т':'t','у':'u','ф':'f','х':'h',
      'ц':'c','ч':'ch','ш':'sh','щ':'sch','ъ':'','ы':'y','ь':'','э':'e',
      'ю':'u','я':'ya', 'А':'A','Б':'B','В':'V','Г':'G','Д':'D','Е':'E','Ё':'YO',
      'Ж':'ZH','З':'Z','И':'I','Й':'I','К':'K','Л':'L','М':'M','Н':'N',
      'О':'O','П':'P','Р':'R','С':'S','Т':'T','У':'U','Ф':'F','Х':'H',
      'Ц':'C','Ч':'CH','Ш':'SH','Щ':'SCH','Ъ':'','Ы':'y','Ь':'','Э':'E',
      'Ю':'U','Я':'YA',',':'','?':'',' ':'_','~':'','!':'','@':'','#':'',
      '$':'','%':'','^':'','&':'','*':'','(':'',')':'','-':'','=':'','+':'',
      ':':'',';':'','<':'','>':'','\'':'','"':'','\\':'','/':'','№':'',
      '[':'',']':'','{':'','}':'','ґ':'','ї':'', 'є':'','Ґ':'g','Ї':'i',
      'Є':'e', '—':''}

    result = ''
    input = f'{last_name}.{first_name}'
    for char in input:
        char_copy = char.upper()
        if char_copy in TRANSLITERATE_DICT:
            if char.isupper():
                char = TRANSLITERATE_DICT[char_copy].capitalize()
            else:  # inserted
                char = TRANSLITERATE_DICT[char_copy].lower()
        result += char
    return result

Writing gost_translate.py


In [5]:
%%writefile mail.py
import smtplib
from email.message import EmailMessage
from core import config

def message_content_generate(content):
    msg = EmailMessage()
    msg['From'] = config.email_login
    msg['To'] = config.mail_send_to
    msg_content = f'\n    <!DOCTYPE html>\n    <html>\n        <body>\n            <div>\n                {content}\n            </div>\n        </body>\n    </html>\n    '
    msg.set_content(msg_content, subtype='html')
    return msg

def send_to_mail(msg: EmailMessage):
    mail = smtplib.SMTP(config.email_server, 25)
    mail.ehlo()
    mail.send_message(msg=msg)
    mail.close()

def empty_code_alert(empty_user_code):
    if len(empty_user_code) == 0:
        return
    text = []
    for user in empty_user_code:
        text.append(f'<h5>Логин: {user.login}</h5>')
    text = '\n'.join(text)
    text = f'<h4>Следующие учетные записи не имеют присвоенного кода физ. лица</h4>{text}<h6>Если код физ. лица не подразумевается для учетной записиси, впишите 000000</h6>'
    msg = message_content_generate(text)
    msg['Subject'] = 'Учетные записи без кода физ.лица'
    send_to_mail(msg=msg)

def empty_code_alert_group(empty_group_code):
    if len(empty_group_code) == 0:
        return
    text = []
    for group in empty_group_code:
        text.append(f'<h5>DN: {group.dn}</h5>')
    text = '\n'.join(text)
    text = f'<h4>Следующие группы не имеют присвоенного кода </h4>{text}<h6>Если код не подразумевается для группы, впишите 000000</h6>'
    msg = message_content_generate(text)
    msg['Subject'] = 'Группы записи без кода'
    send_to_mail(msg=msg)

def empty_code_alert_ou(empty_ou_code):
    if len(empty_ou_code) == 0:
        return
    text = []
    for ou in empty_ou_code:
        text.append(f'<h5>DN: {ou.dn}</h5>')
    text = '\n'.join(text)
    text = f'<h4>Следующие OU не имеют присвоенного кода </h4>{text}<h6>Если код не подразумевается для OU, впишите 000000</h6>'
    msg = message_content_generate(text)
    msg['Subject'] = 'OU записи без кода'
    send_to_mail(msg=msg)

def fio_change_alert(user, new_login, old_fio, new_fio, error_code):
    if error_code == None:
        text = f'<h4>Произошла смена ФИО учетной записи {user.login}</h4><h4>{old_fio} > {new_fio}</h4><h5>Предпологаемый новый логин: {new_login}</h5><h6>Логин не был изменен, смена производится вручную с уведомлением пользователя</h6>'
    else:
        text = f'<h4>Не произошла смена ФИО учетной записи {user.login}</h4><h4>{old_fio} > {new_fio}</h4><h5>Предпологаемый новый логин: {new_login}</h5><h6>{error_code}</h6>'
    msg = message_content_generate(text)
    if error_code == None:
        msg['Subject'] = f'Смена ФИО {user.login}'
    else:
        msg['Subject'] = f'Смена ФИО {user.login} [ОШИБКА]'
    send_to_mail(msg=msg)

def post_change_alert(user, new_post):
    text = f'<h4>Произошла смена должности учетной записи {user.login}</h4><h4>{user.post} > {new_post}</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = f'Смена должности {user.login}'
    send_to_mail(msg=msg)

def divison_not_found_alert(empty_divisions):
    if len(empty_divisions) == 0:
        return
    text = []
    for item in empty_divisions:
        text_tmp = f'<h5>Название: {item.division}, Код: {item.division_code}</h5>'
        if text_tmp not in text:
            text.append(text_tmp)
    text = '\n'.join(text)
    text = f'<h4>Не найдены подразделения в AD:</h4>{text}'
    msg = message_content_generate(text)
    msg['Subject'] = 'Отутствуют подразделения'
    send_to_mail(msg=msg)

def ou_not_found_alert(user, ou):
    text = f'<h4>Не найден OU для учетной записи {user.login}</h4><h4>{ou}</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = f'Смена OU {user.login}'
    send_to_mail(msg=msg)

def division_change_alert(ad_user, file_user):
    old_dn = ','.join(ad_user.dn.split(',')[1:])
    text = f'<h4>Подразделение для учетной записи {ad_user.login} изменено на {file_user.division}</h4><h4>Удалена группа: {ad_user.division_OU.main_group}</h4><h4>Добавлена группа: {file_user.division_OU.main_group}</h4><h4>Перемещен из: {old_dn}</h4><h4>Перемещен в: {file_user.division_OU.dn}</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = f'Смена подразделения {ad_user.login}'
    send_to_mail(msg=msg)

def departament_change_alert(ad_user, file_user, removed_groups):
    text = []
    for remove_group in removed_groups:
        text.append(f'<h4>{remove_group}</h4>')
    text = '\n'.join(text)
    text = f'<h4>Отдел для учетной записи {ad_user.login} изменен на {file_user.division}</h4><h4>Удалены группы:</h4>{text}<h4>Старый отдел: {ad_user.departament_OU.dn}</h4><h4>Новый отдел: {file_user.departament_OU.dn}</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = f'Смена отдела {ad_user.login}'
    send_to_mail(msg=msg)

def disable_users_alert(ad_user):
    text = f'<h4>Учетная запись {ad_user.login} отключена в связи с увольнением</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = f'Учетная запись отключена {ad_user.login}'
    send_to_mail(msg=msg)

def send_error(error):
    text = f'<h4>{error}</h4>'
    msg = message_content_generate(text)
    msg['Subject'] = 'ОШИБКА!!!'
    send_to_mail(msg=msg)

Writing mail.py


In [4]:
%%writefile main.py

from core import logging, config
from mail import empty_code_alert, empty_code_alert_group, empty_code_alert_ou, divison_not_found_alert, disable_users_alert, send_error
from onec import import_users_from_file
from ad import import_users_from_domain, update_FIO, update_company, update_post, update_number, get_all_ou, get_all_groups, departament_change, division_change, disable_user
import traceback
from ldap3 import SAFE_SYNC, Connection, Server, SUBTREE, ALL_ATTRIBUTES, ServerPool, MODIFY_REPLACE

def update_code(user):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        c.modify(user.dn, {'info': [(MODIFY_REPLACE, [0])]})
        logging.debug(c.result)
    return True

def main():
    ad_groups = get_all_groups()
    empty_group_code = []
    for ad_group in ad_groups:
        if ad_group.code == None:
            empty_group_code.append(ad_group)
            logging.info(f'Найдена группа без кода {ad_group.dn}')
    empty_code_alert_group(empty_group_code=empty_group_code)
    ad_ous = get_all_ou(ad_gropus=ad_groups)
    empty_ou_code = []
    for ad_ou in ad_ous:
        if ad_ou.code == None:
            empty_ou_code.append(ad_ou)
            logging.info(f'Найдена OU без кода {ad_ou.dn}')
    empty_code_alert_ou(empty_ou_code=empty_ou_code)
    file_users = import_users_from_file(ad_ous=ad_ous)
    ad_users = import_users_from_domain(ad_ous=ad_ous)
    empty_user_code = []
    for ad_user in ad_users:
        if ad_user.code == None and ad_user.is_lock == False:
            empty_user_code.append(ad_user)
            logging.info(f'Найден AD юзер без кода {ad_user.dn}')
    empty_code_alert(empty_user_code=empty_user_code)
    empty_divisions = []
    for file_user in file_users:
        if file_user.division_OU == None:
            empty_divisions.append(file_user)
            logging.info(f'Найден 1С юзер без подразделения {file_user.FIO}')
    divison_not_found_alert(empty_divisions=empty_divisions)
    for file_user in file_users:
        for ad_user in ad_users:
            if file_user.director_code == ad_user.code:
                file_user.director_obj = ad_user
                break
    for ad_user in ad_users:
        for director_ad_user in ad_users:
            if ad_user.director_dn == director_ad_user.dn:
                ad_user.director_obj = director_ad_user
                break
    for ad_user in ad_users:
        if ad_user.code == 0:
            continue
        found_flag = False
        for file_user in file_users:
            if file_user.code == ad_user.code:
                found_flag = True
                break
        if found_flag == False:
            logging.warning(f'User not found in ad{ad_user.dn}')
            continue
        if ad_user.first_name != file_user.first_name or ad_user.last_name != file_user.last_name or ad_user.middle_name != file_user.middle_name:
            ad_user = update_FIO(ad_user=ad_user, file_user=file_user)
        if ad_user.company != 'АО "Электроавтоматика"':
            update_company(ad_user=ad_user)
        if ad_user.post != file_user.post:
            ad_user = update_post(ad_user=ad_user, new_post=file_user.post)
        if ad_user.number != file_user.number and ad_user.number != None and (file_user.number != ''):
            ad_user = update_number(ad_user=ad_user, new_number=file_user.number)
        if ad_user.departament_OU != file_user.departament_OU:
            ad_user = departament_change(ad_user=ad_user, file_user=file_user)
        if ad_user.division_OU != file_user.division_OU or ad_user.division == None or ad_user.director_obj == None or (ad_user.director_obj != file_user.director_obj):
            ad_user = division_change(ad_user=ad_user, file_user=file_user)
        if ad_user.is_lock == False and file_user.works == False:
            disable_user(user=ad_user)
            disable_users_alert(ad_user=ad_user)
if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        send_error(error=traceback.format_exc())
        logging.critical(e, exc_info=True)
    logging.info('Успешный экспорт')

Writing main.py


In [7]:
%%writefile objects.py

from core import logging

class AD_group_object(object):

    def __init__(self, dn: str, code: int) -> None:
        self.dn = dn
        self.code = code
        logging.info(f'Новая группа: {self.dn}, {self.code}')

    def __str__(self):
        return self.dn

class AD_OU_object(object):

    def __init__(self, dn: str, code: int, main_group: AD_group_object) -> None:
        self.dn = dn
        self.code = code
        self.main_group = main_group
        logging.info(f'Новый OU: {self.dn}, {self.code}')

    def __str__(self):
        return self.dn

class AD_user_object(object):

    def __init__(self, description: str, UserAccountControl: int, director_dn: str, division_OU: AD_OU_object, departament_OU: AD_OU_object, first_name: str, last_name: str, middle_name: str, code: int, director_first_name: str, director_last_name: str, director_middle_name: str, division: str, departament: str, post: str, number: str, is_lock: bool, groups: list, company: str, login: str, dn: str, director_obj=None):
        self.first_name = first_name
        self.last_name = last_name
        self.middle_name = middle_name
        self.code = code
        self.director_first_name = director_first_name
        self.director_last_name = director_last_name
        self.director_middle_name = director_middle_name
        self.division = division
        self.departament = departament
        self.post = post
        self.number = number
        self.is_lock = is_lock
        self.groups = groups
        self.company = company
        self.login = login
        self.dn = dn
        self.FIO = f'{last_name} {first_name} {middle_name}'
        self.division_OU = division_OU
        self.departament_OU = departament_OU
        self.director_obj = director_obj
        self.director_dn = director_dn
        self.UserAccountControl = UserAccountControl
        self.description = self.get_user_description(description)
        logging.info(f'Новый пользователь в AD: {self.dn}, {self.code}')

    def get_user_description(self, description):
        if description == None:
            return ''
        return ' '.join(description)

class File_user_object(object):

    def __init__(self, division_OU: AD_OU_object, departament_OU: AD_OU_object, departament_code: int, division_code: int, director_code: int, first_name: str, last_name: str, middle_name: str, code: int, director_first_name: str, director_last_name: str, director_middle_name: str, division: str, departament: str, post: str, number: str, works: bool, director_obj: AD_user_object=None):
        self.first_name = first_name
        self.last_name = last_name
        self.middle_name = middle_name
        self.code = code
        self.director_first_name = director_first_name
        self.director_last_name = director_last_name
        self.director_middle_name = director_middle_name
        self.division = division
        self.departament = departament
        self.post = post
        self.number = number
        self.works = works
        self.departament_code = departament_code
        self.division_code = division_code
        self.director_code = director_code
        self.FIO = f'{last_name} {first_name} {middle_name}'
        self.division_OU = division_OU
        self.departament_OU = departament_OU
        self.director_obj = director_obj
        logging.info(f'Новый пользователь в 1С: {self.FIO}, {self.code}')

Writing objects.py


In [8]:
%%writefile ad.py
from core import config, logging
from objects import AD_user_object, AD_group_object, AD_OU_object
from gost_translate import gost_translate
from mail import fio_change_alert, post_change_alert, division_change_alert, departament_change_alert
from ldap3 import SAFE_SYNC, Connection, Server, SUBTREE, ALL_ATTRIBUTES, ServerPool, MODIFY_REPLACE
from ldap3.extend.microsoft.addMembersToGroups import ad_add_members_to_groups
from ldap3.extend.microsoft.removeMembersFromGroups import ad_remove_members_from_groups
from typing import List, Optional, Dict, Any

def import_users_from_domain(ad_ous: List[AD_OU_object]) -> List[AD_user_object]:
    logging.info('Searching for users in domain')
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')

    users = []
    try:
        with Connection(server_pool, config.ldap_user, config.ldap_password,
                      client_strategy=SAFE_SYNC, auto_bind=True) as conn:

            results = conn.extend.standard.paged_search(
                search_base=config.ldap_bind,
                search_filter='(&(objectclass=user)(!(objectclass=computer)))',
                search_scope=SUBTREE,
                attributes=ALL_ATTRIBUTES,
                paged_size=500
            )

            for entry in results:
                try:
                    if 'dn' not in entry:
                        continue

                    attrs = entry.get('attributes', {})

                    # Parse name components
                    full_name = attrs.get('cn', '').split()
                    name_parts = {
                        'first_name': full_name[0] if len(full_name) > 0 else None,
                        'middle_name': full_name[1] if len(full_name) > 1 else None,
                        'last_name': full_name[2] if len(full_name) > 2 else None
                    }

                    # Parse manager info
                    manager_data = _parse_manager_info(attrs.get('manager'))

                    # Parse postal code
                    try:
                        code = int(attrs.get('postalCode', 0))
                    except (ValueError, TypeError):
                        code = None

                    # Determine account status
                    user_control = attrs.get('userAccountControl')
                    is_locked = user_control not in [512, 66048] if user_control is not None else False

                    # Find organizational units
                    division_ou, departament_ou = _find_organizational_units(
                        entry['dn'],
                        ad_ous
                    )

                    users.append(AD_user_object(
                        UserAccountControl=user_control,
                        description=attrs.get('description'),
                        director_dn=manager_data['dn'],
                        division_OU=division_ou,
                        departament_OU=departament_ou,
                        dn=entry['dn'],
                        groups=attrs.get('memberOf', []),
                        **name_parts,
                        code=code,
                        director_first_name=manager_data['first_name'],
                        director_last_name=manager_data['last_name'],
                        director_middle_name=manager_data['middle_name'],
                        division=attrs.get('department'),
                        departament=_get_departament_from_dn(attrs.get('distinguishedName')),
                        post=attrs.get('title'),
                        number=attrs.get('telephoneNumber'),
                        is_lock=is_locked,
                        company=attrs.get('company'),
                        login=attrs.get('sAMAccountName')
                    ))
                except Exception as e:
                    logging.error(f'Error processing user {entry.get("dn")}: {str(e)}')
                    continue

    except Exception as e:
        logging.critical(f'AD connection error: {str(e)}')
        raise

    logging.info(f'Found {len(users)} users')
    return users

def _parse_manager_info(manager_dn: Optional[str]) -> Dict[str, Optional[str]]:
    """Parse manager distinguished name into components."""
    if not manager_dn:
        return {
            'dn': None,
            'first_name': None,
            'last_name': None,
            'middle_name': None
        }

    try:
        name_parts = manager_dn.split(',')[0].replace('CN=', '').split()
        return {
            'dn': manager_dn,
            'first_name': name_parts[0] if len(name_parts) > 0 else None,
            'last_name': name_parts[2] if len(name_parts) > 2 else None,
            'middle_name': name_parts[1] if len(name_parts) > 1 else None
        }
    except Exception:
        return {
            'dn': manager_dn,
            'first_name': None,
            'last_name': None,
            'middle_name': None
        }

def _find_organizational_units(dn: str, ad_ous: List[AD_OU_object]) -> tuple:
    """Find division and departament OUs for user DN."""
    division_ou = None
    departament_ou = None

    dn_parts = dn.split(',')[1:]  # Skip CN part
    division_path = ','.join(dn_parts)

    # Find division OU (direct parent)
    division_ou = next((ou for ou in ad_ous if ou.dn == division_path), None)

    # Find departament OU (first OU in path)
    for part in dn_parts:
        if part.startswith('OU='):
            departament_name = part[3:]
            departament_ou = next((ou for ou in ad_ous if ou.dn.endswith(departament_name)), None)
            if departament_ou:
                break

    return division_ou, departament_ou

def _get_departament_from_dn(dn: Optional[str]) -> Optional[str]:
    """Extract departament name from distinguished name."""
    if not dn:
        return None

    for part in dn.split(','):
        if part.startswith('OU='):
            return part[3:]
    return None

def update_FIO(ad_user: AD_user_object, file_user: AD_user_object) -> AD_user_object:
    if not all([file_user.first_name, file_user.last_name]):
        raise ValueError("First name and last name are required")

    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')

    old_fio = ad_user.FIO
    new_cn = f'{file_user.first_name} {file_user.middle_name or ""} {file_user.last_name}'.strip()
    new_dn = f'CN={new_cn},{",".join(ad_user.dn.split(",")[1:])}'

    try:
        with Connection(server_pool, config.ldap_user, config.ldap_password,
                      client_strategy=SAFE_SYNC, auto_bind=True) as conn:

            # Update attributes
            modify_result = conn.modify(ad_user.dn, {
                'displayName': [(MODIFY_REPLACE, [new_cn])],
                'givenName': [(MODIFY_REPLACE, [file_user.first_name])],
                'sn': [(MODIFY_REPLACE, [file_user.last_name])],
                'initials': [(MODIFY_REPLACE, [file_user.middle_name or ""])]
            })

            if not modify_result:
                raise Exception(f"Attribute update failed: {conn.result}")

            # Rename DN if name changed
            if new_cn != ad_user.dn.split(',')[0][3:]:
                conn.modify_dn(ad_user.dn, new_cn)
                if conn.result['result'] != 0:
                    raise Exception(f"DN rename failed: {conn.result}")

            # Update local object
            ad_user.dn = new_dn
            ad_user.first_name = file_user.first_name
            ad_user.middle_name = file_user.middle_name
            ad_user.last_name = file_user.last_name

            # Generate new login and send notification
            new_login = gost_translate(
                first_name=ad_user.first_name,
                middle_name=ad_user.middle_name,
                last_name=ad_user.last_name
            )

            fio_change_alert(
                user=ad_user,
                new_login=new_login,
                old_fio=old_fio,
                new_fio=ad_user.FIO,
                error_code=None
            )

    except Exception as e:
        logging.error(f"Failed to update FIO for {ad_user.dn}: {str(e)}")
        fio_change_alert(
            user=ad_user,
            new_login=None,
            old_fio=old_fio,
            new_fio=f'{file_user.last_name} {file_user.first_name} {file_user.middle_name or ""}'.strip(),
            error_code=str(e)
        )
        raise

    return ad_user

def update_company(ad_user):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        c.modify(ad_user.dn, {'company': [(MODIFY_REPLACE, ['АО \"Электроавтоматика\"'])]})
        logging.debug(c.result)

def update_post(ad_user, new_post):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        c.modify(ad_user.dn, {'title': [(MODIFY_REPLACE, [new_post])]})
        logging.debug(c.result)
    post_change_alert(user=ad_user, new_post=new_post)
    ad_user.post = new_post
    return ad_user

def update_number(ad_user, new_number):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        if new_number == '':
            c.modify(ad_user.dn, {'telephoneNumber': [(MODIFY_REPLACE, [])]})
        else:  # inserted
            c.modify(ad_user.dn, {'telephoneNumber': [(MODIFY_REPLACE, [new_number])]})
        logging.debug(c.result)
    ad_user.number = new_number
    return ad_user

def get_all_groups() -> List[AD_group_object]:
    """Retrieve all groups from Active Directory."""
    logging.info('Retrieving all AD groups')
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')

    groups = []
    try:
        with Connection(server_pool, config.ldap_user, config.ldap_password,
                      client_strategy=SAFE_SYNC, auto_bind=True) as conn:

            results = conn.extend.standard.paged_search(
                search_base=config.ldap_bind,
                search_filter='(objectCategory=Group)',
                search_scope=SUBTREE,
                attributes=['dn', 'info'],
                paged_size=500
            )

            for entry in results:
                try:
                    if 'dn' not in entry:
                        continue

                    # Parse group code
                    try:
                        code = int(entry['attributes'].get('info', 0))
                    except (ValueError, TypeError):
                        code = None

                    groups.append(AD_group_object(
                        dn=entry['dn'],
                        code=code
                    ))
                except Exception as e:
                    logging.warning(f'Error processing group {entry.get("dn")}: {str(e)}')
                    continue

    except Exception as e:
        logging.critical(f'AD connection error: {str(e)}')
        raise

    logging.info(f'Found {len(groups)} groups')
    return groups

def get_all_ou(ad_gropus):
    logging.info('Ищем OU в домене')
    server = Server(config.ldap_server)
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        results = c.extend.standard.paged_search(search_base=config.ldap_bind, search_filter='(objectCategory=organizationalUnit)', search_scope=SUBTREE, attributes=ALL_ATTRIBUTES)
    items = []
    for item in results:
        if 'dn' in item:
            try:
                code = int(item['attributes'].get('postalCode'))
            except:
                code = None
            dn = item['dn']
            ad_group_tmp = None
            for ad_group in ad_gropus:
                if ad_group.code == code and code!= 0:
                    ad_group_tmp = ad_group
                    break
            items.append(AD_OU_object(dn=dn, code=code, main_group=ad_group_tmp))
    logging.info(f'Найдено {len(items)} OU')
    return items
def division_change(ad_user, file_user):
    ad_user_cn = ad_user.dn.split(',')[0]
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        c.modify(ad_user.dn, {
            'department': [(MODIFY_REPLACE, [file_user.division])],
            'manager': [(MODIFY_REPLACE, [file_user.director_obj.dn])]
        })
        # Проверяем, что ad_user.division_OU не None перед обращением к main_group
        if ad_user.division_OU is not None and ad_user.division_OU.main_group is not None:
            ad_remove_members_from_groups(connection=c, members_dn=ad_user.dn, groups_dn=str(ad_user.division_OU.main_group), fix=True)
        # Проверяем, что file_user.division_OU не None перед обращением к main_group
        if file_user.division_OU is not None and file_user.division_OU.main_group is not None:
            ad_add_members_to_groups(connection=c, members_dn=ad_user.dn, groups_dn=str(file_user.division_OU.main_group))
        c.modify_dn(ad_user.dn, ad_user_cn, new_superior=file_user.division_OU.dn)
        logging.debug(c.result)

    division_change_alert(ad_user=ad_user, file_user=file_user)
    ad_user.dn = f'{ad_user_cn},{file_user.division_OU.dn}'
    ad_user.division = file_user.division
    ad_user.director_obj = file_user.director_obj
    ad_user.director_dn = file_user.director_obj.dn
    ad_user.director_first_name = file_user.director_first_name
    ad_user.director_last_name = file_user.director_last_name
    ad_user.director_middle_name = file_user.director_middle_name

    # Проверяем, что ad_user.division_OU не None перед обращением к main_group
    if ad_user.division_OU is not None and ad_user.division_OU.main_group is not None:
        try:
            ad_user.groups.remove(ad_user.division_OU.main_group.dn)
        except:
            pass
    ad_user.division_OU = file_user.division_OU
    return ad_user

def get_user_groups(ad_user):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        results = c.extend.standard.paged_search(search_base=ad_user.dn, search_filter='(&(objectclass=user)(!(objectclass=computer)))', search_scope=SUBTREE, attributes=ALL_ATTRIBUTES)
        for item in results:
            user_groups = item['attributes'].get('memberOf')
            return user_groups

def remove_ad_groups(ad_user, user_groups):
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        tt = ad_remove_members_from_groups(connection=c, members_dn=ad_user.dn, groups_dn=user_groups, fix=True)

def departament_change(ad_user, file_user):
    if ad_user.division_OU!= file_user.division_OU:
        ad_user = division_change(ad_user=ad_user, file_user=file_user)
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')
    with Connection(server_pool, config.ldap_user, config.ldap_password, client_strategy=SAFE_SYNC, auto_bind=True) as c:
        c.modify(ad_user.dn, {'department': [(MODIFY_REPLACE, [file_user.division])], 'manager': [(MODIFY_REPLACE, [file_user.director_obj.dn])]})
        logging.debug(c.result)
    removed_groups = []
    for user_group in ad_user.groups:
        if user_group in config.ignore_on_departament_change_dn_group or user_group == file_user.division_OU.main_group.dn:
            continue
        removed_groups.append(user_group)
    for remove_group in removed_groups:
        ad_user.groups.remove(remove_group)
    remove_ad_groups(ad_user=ad_user, user_groups=removed_groups)
    departament_change_alert(ad_user=ad_user, file_user=file_user, removed_groups=removed_groups)
    return ad_user

def disable_user(user: AD_user_object) -> bool:
    """Disable user account in AD."""
    logging.info(f'Отключаем пользователя {user.login} {user.dn}')
    __CONNECTION_TIMEOUT = 10
    hosts = [config.ldap_server]
    server_pool = ServerPool([Server(host, connect_timeout=__CONNECTION_TIMEOUT) for host in hosts], 'RANDOM')

    # Prepare new values
    new_description = f'{user.description or ""} Уволен'.strip()
    lock = 514 if user.UserAccountControl in [512, 66048] else 66050

    with Connection(server_pool, config.ldap_user, config.ldap_password,
                  client_strategy=SAFE_SYNC, auto_bind=True) as c:

        c.modify(user.dn, {
            'UserAccountControl': [(MODIFY_REPLACE, [lock])],
            'description': [(MODIFY_REPLACE, [new_description])]
        })
        logging.debug(c.result)

    return c.result['result'] == 0

Overwriting ad.py


In [9]:
%%writefile conftest.py
import pytest
import json
import os
from unittest.mock import patch, MagicMock
from core import config

@pytest.fixture
def mock_config(tmp_path):
    # Создаем временный config.json
    config_data = {
        "ldap_server": "ldap://test.local",
        "ldap_user": "test_user",
        "ldap_password": "test_pass",
        "ldap_bind": "dc=test,dc=local",
        "path_to_export_file": str(tmp_path / "export.json"),
        "mail_send_to": "test@example.com"
    }

    config_path = tmp_path / "config.json"
    with open(config_path, 'w') as f:
        json.dump(config_data, f)

    # Патчим конфиг
    with patch('core.config.Config.read_conf', return_value=config_data):
        yield config_data

@pytest.fixture
def sample_export_data(tmp_path):
    data = [{
        "ФИО": "Иванов Иван Иванович",
        "Руководитель": "Петров Петр Петрович",
        "Код": 1001,
        "ПодразделениеУчасток": "IT",
        "Подразделение": "Разработка",
        "Должность": "Программист",
        "Телефон": "+71234567890",
        "Статус": "работает",
        "КодПодразделения": 2001,
        "КодПодразделенияУчастка": 3001,
        "КодРуководителя": 1002
    }]

    export_path = tmp_path / "export.json"
    with open(export_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)

    return export_path

Writing conftest.py


In [20]:
%%writefile test_objects.py
import unittest
from objects import AD_user_object, File_user_object, AD_OU_object, AD_group_object

class TestADObjects(unittest.TestCase):
    def test_ad_user_object(self):
        user = AD_user_object(
            dn="CN=test,OU=Users,DC=test,DC=local",
            login="testuser",
            first_name="Иван",
            last_name="Иванов",
            middle_name="Иванович",
            code=1001,
            is_lock=False,
            groups=[],
            company="Test",
            director_dn=None,
            division_OU=None,
            departament_OU=None,
            director_first_name=None,
            director_last_name=None,
            director_middle_name=None,
            division="IT",
            departament="Dev",
            post="Developer",
            number="123",
            UserAccountControl=512,
            description="Test user"
        )

        self.assertEqual(user.FIO, "Иванов Иван Иванович")
        self.assertEqual(user.is_lock, False)
        self.assertEqual(user.login, "testuser")

    def test_file_user_object(self):
        user = File_user_object(
            first_name="Иван",
            last_name="Иванов",
            middle_name="Иванович",
            code=1001,
            director_first_name="Петр",
            director_last_name="Петров",
            director_middle_name="Петрович",
            division="IT",
            departament="Dev",
            post="Developer",
            number="123",
            works=True,
            departament_code=2001,
            division_code=3001,
            director_code=1002,
            division_OU=None,
            departament_OU=None,
            director_obj=None
        )

        self.assertEqual(user.FIO, "Иванов Иван Иванович")
        self.assertEqual(user.works, True)
        self.assertEqual(user.director_code, 1002)

if __name__ == '__main__':
    unittest.main()

Writing test_objects.py


In [10]:
%%writefile test_onec.py
import unittest
import json
import os
from unittest.mock import patch, mock_open
from onec import import_users_from_file
from objects import AD_OU_object

class TestOneCImport(unittest.TestCase):
    @patch('builtins.open', new_callable=mock_open, read_data=json.dumps([
        {
            "ФИО": "Иванов Иван Иванович",
            "Руководитель": "Петров Петр Петрович",
            "Код": 1001,
            "ПодразделениеУчасток": "IT",
            "Подразделение": "Разработка",
            "Должность": "Программист",
            "Телефон": "+71234567890",
            "Статус": "работает",
            "КодПодразделения": 2001,
            "КодПодразделенияУчастка": 3001,
            "КодРуководителя": 1002
        }
    ]))
    def test_import_users_from_file(self, mock_file):
        ad_ous = [
            AD_OU_object(dn="OU=IT,DC=test,DC=local", code=3001, main_group=None),
            AD_OU_object(dn="OU=Dev,DC=test,DC=local", code=2001, main_group=None)
        ]

        users = import_users_from_file(ad_ous)

        self.assertEqual(len(users), 1)
        self.assertEqual(users[0].FIO, "Иванов Иван Иванович")
        self.assertEqual(users[0].division, "IT")
        self.assertEqual(users[0].works, True)

    def test_empty_file(self):
        with patch('builtins.open', mock_open(read_data=json.dumps([]))):
            users = import_users_from_file([])
            self.assertEqual(len(users), 0)

if __name__ == '__main__':
    unittest.main()

Writing test_for_onec.py


In [19]:
%%writefile test_ad.py
import unittest
from unittest.mock import patch, MagicMock
from ldap3 import MODIFY_REPLACE
from ad import import_users_from_domain, update_FIO, disable_user
from objects import AD_user_object, File_user_object

class TestADFunctions(unittest.TestCase):
    @patch('ad.ServerPool')
    @patch('ad.Connection')
    def test_import_users_from_domain(self, mock_conn, mock_pool):
        # Настраиваем моки
        mock_conn.return_value.__enter__.return_value.extend.standard.paged_search.return_value = [
            {
                'dn': 'CN=test,OU=Users,DC=test,DC=local',
                'attributes': {
                    'cn': 'Иванов Иван Иванович',
                    'sAMAccountName': 'testuser',
                    'memberOf': ['CN=Group1,OU=Groups,DC=test,DC=local'],
                    'department': 'IT',
                    'company': 'Test',
                    'title': 'Developer',
                    'telephoneNumber': '123',
                    'userAccountControl': 512,
                    'description': 'Test user'
                }
            }
        ]

        users = import_users_from_domain([])

        self.assertEqual(len(users), 1)
        self.assertEqual(users[0].login, "testuser")
        self.assertEqual(users[0].is_lock, False)

    @patch('ad.ServerPool')
    @patch('ad.Connection')
    def test_update_FIO(self, mock_conn, mock_pool):
        # Настраиваем мок соединения
        mock_connection = MagicMock()
        mock_connection.result = {'result': 0}
        mock_conn.return_value.__enter__.return_value = mock_connection

        ad_user = AD_user_object(
            dn="CN=Иванов Иван Иванович,OU=Users,DC=test,DC=local",
            login="ivanov",
            first_name="Иван",
            last_name="Иванов",
            middle_name="Иванович",
            code=1001,
            is_lock=False,
            groups=[],
            company="Test",
            director_dn=None,
            division_OU=None,
            departament_OU=None,
            director_first_name=None,
            director_last_name=None,
            director_middle_name=None,
            division="IT",
            departament="Dev",
            post="Developer",
            number="123",
            UserAccountControl=512,
            description="Test user"
        )

        file_user = File_user_object(
            first_name="Иван",
            last_name="Сидоров",
            middle_name="Иванович",
            code=1001,
            is_lock=False,
            groups=[],
            company="Test",
            director_dn=None,
            division_OU=None,
            departament_OU=None,
            director_first_name=None,
            director_last_name=None,
            director_middle_name=None,
            division="IT",
            departament="Dev",
            post="Developer",
            number="123",
            UserAccountControl=512,
            description="Test user"
        )

        updated_user = update_FIO(ad_user, file_user)

        self.assertEqual(updated_user.FIO, "Сидоров Иван Иванович")
        self.assertEqual(updated_user.last_name, "Сидоров")

    @patch('ad.ServerPool')
    @patch('ad.Connection')
    def test_disable_user_error(self, mock_conn, mock_pool):
        # Настраиваем мок с ошибкой
        mock_connection = MagicMock()
        mock_connection.result = {'result': 1, 'description': 'Error'}
        mock_conn.return_value.__enter__.return_value = mock_connection

        user = AD_user_object(
            dn="CN=test,OU=Users,DC=test,DC=local",
            login="testuser",
            first_name="Иван",
            last_name="Иванов",
            middle_name="Иванович",
            code=1001,
            is_lock=False,
            groups=[],
            company="Test",
            director_dn=None,
            division_OU=None,
            departament_OU=None,
            director_first_name=None,
            director_last_name=None,
            director_middle_name=None,
            division="IT",
            departament="Dev",
            post="Developer",
            number="123",
            UserAccountControl=512,
            description="Test user"
        )

        result = disable_user(user)
        self.assertFalse(result)

if __name__ == '__main__':
    unittest.main()

Writing test_for_ad.py


In [22]:
%%writefile test_main_flow.py
import unittest
from unittest.mock import patch, MagicMock
from main import main
from core import config

class TestMainFlow(unittest.TestCase):
    @patch('main.get_all_groups')
    @patch('main.get_all_ou')
    @patch('main.import_users_from_file')
    @patch('main.import_users_from_domain')
    @patch('main.update_FIO')
    @patch('main.update_company')
    @patch('main.update_post')
    @patch('main.update_number')
    @patch('main.departament_change')
    @patch('main.division_change')
    @patch('main.disable_user')
    def test_main_flow(self, mock_disable, mock_div_change, mock_dept_change,
                      mock_update_num, mock_update_post, mock_update_comp,
                      mock_update_fio, mock_import_ad, mock_import_file,
                      mock_get_ou, mock_get_groups):
        # Настраиваем моки
        mock_get_groups.return_value = []
        mock_get_ou.return_value = []

        # Моки для пользователей
        ad_user = MagicMock()
        ad_user.code = 1001
        ad_user.is_lock = False
        ad_user.first_name = "Иван"
        ad_user.last_name = "Иванов"
        ad_user.middle_name = "Иванович"
        ad_user.post = "Старая должность"
        ad_user.number = "123"
        ad_user.company = "Old Company"

        file_user = MagicMock()
        file_user.code = 1001
        file_user.first_name = "Иван"
        file_user.last_name = "Иванов"
        file_user.middle_name = "Иванович"
        file_user.post = "Новая должность"
        file_user.number = "456"
        file_user.works = True

        mock_import_ad.return_value = [ad_user]
        mock_import_file.return_value = [file_user]

        main()

        mock_update_fio.assert_called()
        mock_update_comp.assert_called()
        mock_update_post.assert_called_with(ad_user, "Новая должность")
        mock_update_num.assert_called_with(ad_user, "456")

if __name__ == '__main__':
    unittest.main()

Writing test_main_flow.py


In [ ]:
%%writefile run_tests.py
import unittest
loader = unittest.TestLoader()
suite = loader.discover('test')
runner = unittest.TextTestRunner(verbosity=2)
result = runner.run(suite)
if not result.wasSuccessful():
  exit(1)